In [1]:
%run /home/bb/src/python/valkyrie/apps/common/run_notebook_import.py

/media/veracrypt1/python/valkyrie/apps/common/run_notebook_import.py:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


/media/veracrypt1/python/valkyrie/lib/valkyrie/ml/data.py:291: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [2]:
import copy
from itertools import product as cartesian_product

from overrides import overrides
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
from torch.nn import functional as F

from d2l.torch import init_cnn

from valkyrie.tools import *
from valkyrie.ml import modules
from valkyrie.ml.utils import tensor, HyperParameters
from valkyrie.ml import utils as ml_utils
from valkyrie.ml.data import Df2T2, SubDataSet
from valkyrie.ml.net import LrNet, AlexRegNet
from valkyrie.ml.net import Regressor

from valkyrie.nibelungen.data import DataMgr

In [ ]:
from valkyrie.ml.data import DataModule

sdate, edate = '2023-01-01', '2023-06-30'
instrs = ['BDM.BTC.USDT.FP']
freq = '1s'
ret_n_s = [120] #[60, 120, 300, 600]
root_dir = '/home/bb/data/BDM'
#########################################################
#DataMgr
#########################################################
data_mgr = DataMgr(sdate, edate, freq, root_dir, instrs = instrs, ret_n_s = ret_n_s)
df_res = data_mgr.get(instrs[0]) 
channels = ['bpx_last','apx_last'] + ['bq_last','aq_last','buy_qty_sum', 'sell_qty_sum'] 
n_channels = len(channels)
H, W = 16, 64
df2t2 = Df2T2(df_res, H, W, xcols = channels, ycol = 'mid_last_ret_120_n', yscaler = 1)

train_set, val_set = SubDataSet.create_train_valiation_pair(df2t2, 0.8)
dm = DataModule.from_dataset(train_set, val_set, 2048)

In [ ]:
testNet = AlexRegNet(lr = 1e-4, num_channels = n_channels, height = H)
X = df2t2[0][0]
ml_utils.layer_summary(testNet, X.shape)
#ml_utils.parameter_summary(testNet)

In [ ]:
trainer = modules.Trainer(max_epochs=100, num_gpus=1)
with Timer('training'):
    trainer.fit(testNet, dm)

In [ ]:
ne = NetEvaluator(testNet, dm)
ne.eval_performance()